# Hit API

In [1]:
from caltrain import (
    ID2NAME,
    replace_colnames,
    convert_time_str_to_local_tz_timestamp,
    MY_TRAIN_TIME_COLS,
    API_TRAIN_TIME_COLS,
    MAP_TO_AVAILABLE_STATION,
    iso_to_timestamp,
)

from caltrain import (
    RwcSfTrains,
    convert_time_str_to_local_tz_timestamp,
    RWC_CALTRAIN_STOP_ID,
    format_for_display,
    one_vehicle_activity_to_stops_with_vehicle_id
)

In [2]:
{os.environ.pop(k) for k in os.environ.keys() if 'PROXY' in k}

{'localhost,.dev.bloomberg.com,query.di.secondmeasure.com'}

In [3]:
self = RwcSfTrains('north')
# self = RwcSfTrains('south')
self.fetch_data()

/Users/ahakso/gd/sandbox/caltrain.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, time_cols] = df.loc[:, time_cols].applymap(iso_to_timestamp)
/Users/ahakso/gd/sandbox/caltrain.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, time_cols] = df.loc[:, time_cols].applymap(iso_to_timestamp)


In [4]:
stop_id = self.my_departure_station_id

In [6]:
all_trains_one_direction = self.get_trains_one_direction_from_departures_response(stop_id)
station_departures = self.convert_predicted_stops_json_to_df(
    all_trains_one_direction,
    ID2NAME[stop_id],
)
station_departures_filtered = station_departures.loc[lambda df: df.stop_name == MAP_TO_AVAILABLE_STATION[ID2NAME[stop_id]]]
station_departures_filtered = convert_time_str_to_local_tz_timestamp(
    station_departures_filtered,
    time_cols=MY_TRAIN_TIME_COLS if np.any(["_" in x for x in station_departures.columns]) else API_TRAIN_TIME_COLS,
)

/Users/ahakso/gd/sandbox/caltrain.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, time_cols] = df.loc[:, time_cols].applymap(iso_to_timestamp)


In [13]:
self.trains_with_departure_stop

,stop_name,vehicle_id,scheduled_departure,expected_departure,time_late
4,Redwood City Caltrain Station,505,2023-06-22 11:18:00-07:00,2023-06-22 11:21:04-07:00,0 days 00:03:04
13,Redwood City Caltrain Station,115,2023-06-22 11:37:00-07:00,2023-06-22 11:37:30-07:00,0 days 00:00:30
37,Redwood City Caltrain Station,507,2023-06-22 12:18:00-07:00,2023-06-22 12:18:00-07:00,0 days 00:00:00
46,Redwood City Caltrain Station,117,2023-06-22 12:37:00-07:00,2023-06-22 12:37:30-07:00,0 days 00:00:30


In [17]:
self.get_next_sf_trips_from_rwc()

trains {'117', '507'} not in the realtime response :thunk:


,stop name,scheduled departure,expected departure,time late,stop name onward stop,scheduled departure onward stop,expected departure onward stop,trip duration from origin
4,Redwood City Caltrain Station,11:18,11:21,3 minutes,San Francisco Caltrain Station,11:57,11:57,36 minutes
21,Redwood City Caltrain Station,11:37,11:37,0 minutes,San Francisco Caltrain Station,12:33,12:33,56 minutes
22,Redwood City Caltrain Station,12:18,12:18,0 minutes,nan,-,-,-nan minutes
23,Redwood City Caltrain Station,12:37,12:37,0 minutes,nan,-,-,-nan minutes


In [16]:
import re

In [31]:
(
    self.trains_with_departure_stop.merge(
        self.get_vehicle_onward_stops(),
        on="vehicle_id",
        how="left",
        suffixes=("_departure", "_destination"),
    ).drop(columns=["vehicle_id"])
).loc[
    lambda df: (
        df.stop_name_departure.apply(lambda x: bool(re.search("Redwood", str(x))))
        & df.stop_name_destination.apply(lambda x: bool(re.search("22nd|4th", str(x))))
    )
    .astype(bool)
]

ValueError: No objects to concatenate

# Misc info gathering

In [13]:
CALTRAIN_OPERATOR_ID = "CT"
# This gives caltrain operatorID: CT
operators = f"http://api.511.org/transit/operators?api_key={os.environ['CALTRAIN_API_KEY']}"
# this gives the name of rwc caltrain stop
stops = f"http://api.511.org/transit/stops?api_key={os.environ['CALTRAIN_API_KEY']}&operator_id={CALTRAIN_OPERATOR_ID}&format=json"

In [17]:
response_stops = RwcSfTrains.request_to_dict(stops)
# [x for x in response_stops.json()['Contents']['dataObjects']['ScheduledStopPoint'] if 'Redwood'.lower() in x['Name'].lower()]

In [24]:
self.departures_response.keys()#['redwood_city']

dict_keys(['redwood_city', 'san_francisco', '22nd_street'])

In [25]:
self.departures_response['san_francisco']

{'ServiceDelivery': {'ResponseTimestamp': '2023-06-08T15:54:04Z',
  'ProducerRef': 'CT',
  'Status': True,
  'StopMonitoringDelivery': {'version': '1.4',
   'ResponseTimestamp': '2023-06-08T15:54:04Z',
   'Status': True,
   'MonitoredStopVisit': [{'RecordedAtTime': '2023-06-08T15:53:52Z',
     'MonitoringRef': '70142',
     'MonitoredVehicleJourney': {'LineRef': 'L4',
      'DirectionRef': 'S',
      'FramedVehicleJourneyRef': {'DataFrameRef': '2023-06-08',
       'DatedVehicleJourneyRef': '406'},
      'PublishedLineName': 'LTD 4',
      'OperatorRef': 'CT',
      'OriginRef': '70012',
      'OriginName': 'San Francisco Caltrain Station',
      'DestinationRef': '70262',
      'DestinationName': 'San Jose Diridon',
      'Monitored': True,
      'InCongestion': None,
      'VehicleLocation': {'Longitude': '-122.234032', 'Latitude': '37.487381'},
      'Bearing': None,
      'Occupancy': None,
      'VehicleRef': '406',
      'MonitoredCall': {'StopPointRef': '70142',
       'StopPoint

In [25]:
response_stops['Contents']['dataObjects']['ScheduledStopPoint'][2]

{'id': '70021',
 'Extensions': {'LocationType': '0',
  'PlatformCode': None,
  'ParentStation': '22nd_street'},
 'Name': '22nd Street Caltrain Station',
 'Location': {'Longitude': '-122.39188', 'Latitude': '37.757599'},
 'Url': None,
 'StopType': 'onstreetBus'}

In [19]:
[x for x in response_stops['Contents']['dataObjects']['ScheduledStopPoint'] if 'San' in x['Name']]

NameError: name 'response_stops' is not defined

In [51]:
self.departures_response["ServiceDelivery"]["StopMonitoringDelivery"]["MonitoredStopVisit"]

[{'RecordedAtTime': '1970-01-01T00:00:00Z',
  'MonitoringRef': '70012',
  'MonitoredVehicleJourney': {'LineRef': 'L3',
   'DirectionRef': 'S',
   'FramedVehicleJourneyRef': {'DataFrameRef': '2023-06-08',
    'DatedVehicleJourneyRef': '302'},
   'PublishedLineName': 'LTD 3',
   'OperatorRef': 'CT',
   'OriginRef': '70012',
   'OriginName': 'San Francisco Caltrain Station',
   'DestinationRef': '70262',
   'DestinationName': 'San Jose Diridon',
   'Monitored': True,
   'InCongestion': None,
   'VehicleLocation': {'Longitude': '', 'Latitude': ''},
   'Bearing': None,
   'Occupancy': None,
   'VehicleRef': None,
   'MonitoredCall': {'StopPointRef': '70012',
    'StopPointName': 'San Francisco Caltrain Station',
    'VehicleLocationAtStop': '',
    'VehicleAtStop': '',
    'DestinationDisplay': 'San Jose Diridon',
    'AimedArrivalTime': '2023-06-08T13:27:00Z',
    'ExpectedArrivalTime': None,
    'AimedDepartureTime': '2023-06-08T13:27:00Z',
    'ExpectedDepartureTime': '2023-06-08T13:27:0

In [47]:
[x for x in response_stops['Contents']['dataObjects']['ScheduledStopPoint'] if 'Redwood' in x['Name']]

[{'id': 'redwood_city',
  'Extensions': {'LocationType': '1',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'Redwood City',
  'Location': {'Longitude': '-122.2315', 'Latitude': '37.485865'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70141',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'redwood_city'},
  'Name': 'Redwood City Caltrain Station',
  'Location': {'Longitude': '-122.231936', 'Latitude': '37.486159'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70142',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'redwood_city'},
  'Name': 'Redwood City Caltrain Station',
  'Location': {'Longitude': '-122.232', 'Latitude': '37.486101'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': 'RC',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'Temporary Stop - Redwood City',
  'Location': {'Longitude': '-122.2320244337', 'Latitude

In [50]:
[x for x in response_stops['Contents']['dataObjects']['ScheduledStopPoint'] if 'San Fr' in x['Name']]

[{'id': 'san_francisco',
  'Extensions': {'LocationType': '1',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'San Francisco Caltrain Station',
  'Location': {'Longitude': '-122.394911', 'Latitude': '37.776404'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70011',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'san_francisco'},
  'Name': 'San Francisco Caltrain Station',
  'Location': {'Longitude': '-122.394992', 'Latitude': '37.77639'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70012',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'san_francisco'},
  'Name': 'San Francisco Caltrain Station',
  'Location': {'Longitude': '-122.394935', 'Latitude': '37.776348'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': 'south_sf',
  'Extensions': {'LocationType': '1',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'South San Francisco Caltrain Station',
  'Location': {'